In [1]:
import sys
import torch 
import os
import random
import torch.nn as nn
import numpy as np
import pandas as pd 
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
from sklearn.metrics import cohen_kappa_score, mean_squared_error, mean_absolute_error
import pickle
import cv2
import torch.nn.functional as F
from torchvision import models
import seaborn as sns
import random
import sys
import collections
from torch.utils.data import Dataset, WeightedRandomSampler, SubsetRandomSampler, DataLoader
from albumentations import (
    HorizontalFlip, VerticalFlip, CenterCrop, RandomRotate90, RandomCrop, 
    PadIfNeeded, Normalize, Flip, OneOf, Compose, Resize, Transpose, 
    IAAAdditiveGaussianNoise, GaussNoise, CLAHE, RandomBrightnessContrast, HueSaturationValue,
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)
from catalyst.contrib.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from catalyst.dl.experiment import SupervisedExperiment
from catalyst.dl.runner import SupervisedRunner
from catalyst.dl.callbacks import EarlyStoppingCallback, MixupCallback,AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.dl.core.state import RunnerState
from catalyst.dl.core import MetricCallback
from catalyst.dl.callbacks import CriterionCallback
from sklearn.model_selection import train_test_split

In [2]:
package_path = '/kaggle/aptos/EfficientNet-PyTorch/efficientnet_pytorch'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [4]:
num_classes = 1
seed_everything(1234)
lr          = 3e-4
IMG_SIZE    = 256

In [5]:
model = EfficientNet.from_pretrained('efficientnet-b7')
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, num_classes)
model.cuda()

Loaded pretrained weights for efficientnet-b7


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_siz

In [6]:
model.load_state_dict(torch.load(os.path.join('logs/efficient_net_b7_regressionpretrain_smart_v1/','checkpoints/last.pth'))['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if (check_shape == 0):  # image is too dark so that we crop out everything,
            return img  # return original image
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1, img2, img3], axis=-1)
        #         print(img.shape)
        return img

In [8]:
class DiabeticDataset(Dataset):
    def __init__(self, dataset_path, labels, ids, albumentations_tr, extens, shuffle=True):
        self.labels = labels
        self.ids = ids
        self.shuffle = shuffle
        self.dataset_path = dataset_path
        self.albumentations_tr = albumentations_tr
        self.extens = extens
            
    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        #imid = self.ids[index]
        image = cv2.imread(os.path.join(self.dataset_path, self.ids[index] + '.{}'.format(self.extens)))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        if self.albumentations_tr:
            augmented = self.albumentations_tr(image=image)
            image = augmented['image']
        target = self.labels[index]
        return torch.from_numpy(image.transpose((2, 0, 1))).float(), torch.tensor(np.expand_dims(target,0)).float()

In [9]:
def quadratic_weighted_kappa(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    threshold: float = None,
    activation: str = None
):
    """
    Args:
        outputs (torch.Tensor): A list of predicted elements
        targets (torch.Tensor):  A list of elements that are to be predicted
        activation (str): An torch.nn activation applied to the outputs.
            Must be one of ["none", "Sigmoid", "Softmax2d"]
    Returns:
        float: quadratic kappa score
    """
    outputs = outputs.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    outputs_clipped = list()
    outputs_clipped = np.rint(outputs)
    outputs_clipped[outputs_clipped<0] = 0
    outputs_clipped[outputs_clipped>4] = 4
    #for o in outputs:
    #    if o <= 0.5:
    #        outputs_clipped.append(0)
    #    if 0.5 > o <= 1.5:
    #        outputs_clipped.append(1)
    #    if 1.5 < o <= 2.5:
    #        outputs_clipped.append(2)
    #    if 2.5 < o <= 3.5:
    #        outputs_clipped.append(3)
    #    if o > 3.5:
    #        outputs_clipped.append(4)      
    #simple clip of outputs
    score = cohen_kappa_score(outputs_clipped, targets, weights='quadratic')
    return score
class QuadraticKappScoreMetricCallback(MetricCallback):
    """
    F1 score metric callback.
    """

    def __init__(
        self,
        input_key: str = "targets",
        output_key: str = "logits",
        prefix: str = "qkappa_score",
        activation: str = None
    ):
        """
        Args:
            input_key (str): input key to use for iou calculation
                specifies our ``y_true``.
            output_key (str): output key to use for iou calculation;
                specifies our ``y_pred``
            activation (str): An torch.nn activation applied to the outputs.
                Must be one of ['none', 'Sigmoid', 'Softmax2d']
        """

        super().__init__(
            prefix=prefix,
            metric_fn=quadratic_weighted_kappa,
            input_key=input_key,
            output_key=output_key,
            activation=activation
        )
        
def mean_squared_error_callback(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    threshold: float = None,
    activation: str = None
):
    """
    Args:
        outputs (torch.Tensor): A list of predicted elements
        targets (torch.Tensor):  A list of elements that are to be predicted
        activation (str): An torch.nn activation applied to the outputs.
            Must be one of ["none", "Sigmoid", "Softmax2d"]
    Returns:
        float: quadratic kappa score
    """

    outputs = outputs.cpu().detach().numpy()
    score = mean_squared_error(outputs, targets.detach().cpu().numpy())
    return score
class MSECallback(MetricCallback):
    """
    F1 score metric callback.
    """

    def __init__(
        self,
        input_key: str = "targets",
        output_key: str = "logits",
        prefix: str = "mse_score",
        activation: str = None
    ):
        """
        Args:
            input_key (str): input key to use for iou calculation
                specifies our ``y_true``.
            output_key (str): output key to use for iou calculation;
                specifies our ``y_pred``
            activation (str): An torch.nn activation applied to the outputs.
                Must be one of ['none', 'Sigmoid', 'Softmax2d']
        """

        super().__init__(
            prefix=prefix,
            metric_fn=mean_squared_error_callback,
            input_key=input_key,
            output_key=output_key,
            activation=activation
        )
def mean_absolute_error_callback(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    threshold: float = None,
    activation: str = None
):
    """
    Args:
        outputs (torch.Tensor): A list of predicted elements
        targets (torch.Tensor):  A list of elements that are to be predicted
        activation (str): An torch.nn activation applied to the outputs.
            Must be one of ["none", "Sigmoid", "Softmax2d"]
    Returns:
        float: quadratic kappa score
    """

    outputs = outputs.cpu().detach().numpy()
    score = mean_absolute_error(outputs, targets.detach().cpu().numpy())
    return score
class MAECallback(MetricCallback):
    """
    F1 score metric callback.
    """

    def __init__(
        self,
        input_key: str = "targets",
        output_key: str = "logits",
        prefix: str = "mae_score",
        activation: str = None
    ):
        """
        Args:
            input_key (str): input key to use for iou calculation
                specifies our ``y_true``.
            output_key (str): output key to use for iou calculation;
                specifies our ``y_pred``
            activation (str): An torch.nn activation applied to the outputs.
                Must be one of ['none', 'Sigmoid', 'Softmax2d']
        """

        super().__init__(
            prefix=prefix,
            metric_fn=mean_absolute_error_callback,
            input_key=input_key,
            output_key=output_key,
            activation=activation
        )
def aug_train(resolution,p=1): 
    return Compose([Resize(resolution,resolution),
                    OneOf([
                        HorizontalFlip(), 
                        VerticalFlip(), 
                        RandomRotate90(), 
                        Transpose()],p=0.5),
                    Normalize()
                    ], p=p)
def aug_val(resolution,p=1): 
    return Compose([Resize(resolution,resolution),Normalize()], p=p)
def strong_aug(resolution, p=1.0):
    return Compose([
        Resize(resolution,resolution),
        OneOf([RandomRotate90(),Flip(),Transpose(),HorizontalFlip(),VerticalFlip()],p=1.0),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.5),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.1),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
        Normalize()
    ], p=p)

In [10]:
train = pd.read_csv('/home/skolchen/kaggle/aptos/train.csv')
test = pd.read_csv('/home/skolchen/kaggle/aptos/test.csv')
sample_submission = pd.read_csv('/home/skolchen/kaggle/aptos/sample_submission.csv')
duplicated_images = pd.read_csv('/home/skolchen/kaggle/aptos/duplicated_info.csv')
duplicated_images['id_code'] = duplicated_images['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
image_infos = pd.read_csv('/home/skolchen/kaggle/aptos/image_info.csv')
image_infos['id_code'] = image_infos['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
train = train.loc[[x not in duplicated_images['id_code'].values for x in train['id_code'].values],:]
X = train['id_code'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, train['diagnosis'].values, test_size=0.1, random_state=42, stratify=train['diagnosis'].values)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [12]:
train_dataset = DiabeticDataset(dataset_path='/home/skolchen/kaggle/aptos/train/', 
                                labels = y_train, 
                                ids = X_train, 
                                albumentations_tr = strong_aug(IMG_SIZE), 
                                extens='png') 
val_dataset = DiabeticDataset(dataset_path='/home/skolchen/kaggle/aptos/train/', 
                              labels = y_val, 
                              ids = X_val, 
                              albumentations_tr = aug_val(IMG_SIZE), 
                              extens='png') 

In [13]:
train_loader =  DataLoader(train_dataset,
                           num_workers=16,
                           pin_memory=False,
                           batch_size=12,
                           shuffle=True)
val_loader = DataLoader(val_dataset,
                        num_workers=16,
                        pin_memory=False,
                        batch_size=12)
loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = val_loader

In [14]:
runner = SupervisedRunner()

In [15]:
logdir = 'logs/efficient_net_b7_trainregression_smart_v1/'

In [ ]:
!rm -r logs/efficient_net_b7_trainregression_v1/

In [16]:
#make cutmix onlyfor same classes
from typing import List
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

class CutmixCallbackSameClasses(CriterionCallback):
    """
    Callback to do mixup augmentation.
    Paper: https://arxiv.org/abs/1710.09412
    Note:
        MixupCallback is inherited from CriterionCallback and
        does its work.
        You may not use them together.
    """

    def __init__(
        self,
        fields: List[str] = ("features", ),
        alpha=1.0,
        on_train_only=True,
        **kwargs
    ):
        """
        Args:
            fields (List[str]): list of features which must be affected.
            alpha (float): beta distribution a=b parameters.
                Must be >=0. The more alpha closer to zero
                the less effect of the mixup.
            on_train_only (bool): Apply to train only.
                As the mixup use the proxy inputs, the targets are also proxy.
                We are not interested in them, are we?
                So, if on_train_only is True, use a standard output/metric
                for validation.
        """
        assert len(fields) > 0, \
            "At least one field for MixupCallback is required"
        assert alpha >= 0, "alpha must be>=0"

        super().__init__(**kwargs)

        self.on_train_only = on_train_only
        self.fields = fields
        self.alpha = alpha
        self.lam = 1
        self.index = list()
        self.is_needed = True

    def on_loader_start(self, state: RunnerState):
        self.is_needed = not self.on_train_only or \
            state.loader_name.startswith("train")

    def on_batch_start(self, state: RunnerState):
        if not self.is_needed:
            return

        if self.alpha > 0:
            self.lam = np.random.beta(self.alpha, self.alpha)
        else:
            self.lam = 1
        
        classes_in_batch = torch.unique(state.input[self.input_key]).cpu().numpy()
        batch_idx = np.arange(state.input[self.fields[0]].shape[0])
        #now make permutations per each class
        
        for idx, image_class in enumerate(classes_in_batch):
            #images with this class
            class_mask = (state.input[self.input_key].cpu().numpy()==image_class).squeeze()
            class_idx = batch_idx[class_mask]
            index = np.random.permutation(class_idx)
            index = torch.tensor(index, dtype=torch.long)
            class_idx = torch.tensor(class_idx, dtype=torch.long)
            index.to(state.device)
            class_idx.to(state.device)
            for f in self.fields:
                bbx1, bby1, bbx2, bby2 = rand_bbox(state.input[f].size(), self.lam)
                state.input[f][class_idx, :, bbx1:bbx2, bby1:bby2] = state.input[f][index, :, bbx1:bbx2, bby1:bby2]

    def _compute_loss(self, state: RunnerState, criterion):
        if not self.is_needed:
            return super()._compute_loss(state, criterion)

        pred = state.output[self.output_key]
        y = state.input[self.input_key]

        loss = self.lam * criterion(pred, y)
        return loss
    
class MixupCallbackSameClass(CriterionCallback):
    """
    Callback to do mixup augmentation.
    Paper: https://arxiv.org/abs/1710.09412
    Note:
        MixupCallback is inherited from CriterionCallback and
        does its work.
        You may not use them together.
    """

    def __init__(
        self,
        fields: List[str] = ("features", ),
        alpha=1.0,
        on_train_only=True,
        **kwargs
    ):
        """
        Args:
            fields (List[str]): list of features which must be affected.
            alpha (float): beta distribution a=b parameters.
                Must be >=0. The more alpha closer to zero
                the less effect of the mixup.
            on_train_only (bool): Apply to train only.
                As the mixup use the proxy inputs, the targets are also proxy.
                We are not interested in them, are we?
                So, if on_train_only is True, use a standard output/metric
                for validation.
        """
        assert len(fields) > 0, \
            "At least one field for MixupCallback is required"
        assert alpha >= 0, "alpha must be>=0"

        super().__init__(**kwargs)

        self.on_train_only = on_train_only
        self.fields = fields
        self.alpha = alpha
        self.lam = 1
        self.index = None
        self.is_needed = True

    def on_loader_start(self, state: RunnerState):
        self.is_needed = not self.on_train_only or \
            state.loader_name.startswith("train")

    def on_batch_start(self, state: RunnerState):
        if not self.is_needed:
            return

        if self.alpha > 0:
            self.lam = np.random.beta(self.alpha, self.alpha)
        else:
            self.lam = 1

        classes_in_batch = torch.unique(state.input[self.input_key]).cpu().numpy()
        batch_idx = np.arange(state.input[self.fields[0]].shape[0])
        #now make permutations per each class
        
        for idx, image_class in enumerate(classes_in_batch):
            #images with this class
            class_mask = (state.input[self.input_key].cpu().numpy()==image_class).squeeze()
            class_idx = batch_idx[class_mask]
            index = np.random.permutation(class_idx)
            index = torch.tensor(index, dtype=torch.long)
            class_idx = torch.tensor(class_idx, dtype=torch.long)
            index.to(state.device)
            class_idx.to(state.device)
            for f in self.fields:
                state.input[f][class_idx] = self.lam * state.input[f][class_idx] + (1 - self.lam) * state.input[f][index]
                
    def _compute_loss(self, state: RunnerState, criterion):
        if not self.is_needed:
            return super()._compute_loss(state, criterion)

        pred = state.output[self.output_key]
        y = state.input[self.input_key]

        loss = self.lam * criterion(pred, y)
        return loss


In [17]:
for p in model.parameters():
    p.requires_grad = False
for p in model._fc.parameters():
    p.requires_grad = True
num_epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()
runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        logdir=logdir,
        callbacks=[
            CutmixCallbackSameClasses(),
            QuadraticKappScoreMetricCallback(),
            MSECallback(),
            MAECallback(),               
                  ],
        num_epochs=num_epochs,
        verbose=True
        )     
for p in model.parameters():
    p.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_epochs = 15
criterion = nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=5)
runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        logdir=logdir,
        scheduler=scheduler,
        callbacks=[
            CutmixCallbackSameClasses(),
            QuadraticKappScoreMetricCallback(),
            MSECallback(),
            MAECallback(),               
            EarlyStoppingCallback(patience=25, metric='loss')
                  ],
        num_epochs=num_epochs,
        verbose=True
        )        

0/5 * Epoch (train): 100% 202/202 [00:50<00:00,  1.83it/s, _timers/_fps=9.121, loss=0.163, mae_score=0.511, mse_score=0.554, qkappa_score=0.643]  
0/5 * Epoch (valid): 100% 51/51 [00:14<00:00,  2.02it/s, _timers/_fps=10.020, loss=0.081, mae_score=0.250, mse_score=0.081, qkappa_score=1.000] 
[2019-08-10 18:54:17,003] 
0/5 * Epoch 0 (train): _base/lr=0.0003 | _base/momentum=0.9000 | _timers/_fps=90.9689 | _timers/batch_time=0.1643 | _timers/data_time=0.0285 | _timers/model_time=0.1333 | loss=0.2978 | mae_score=0.5338 | mse_score=0.6352 | qkappa_score=0.7222
0/5 * Epoch 0 (valid): _base/lr=0.0003 | _base/momentum=0.9000 | _timers/_fps=123.7166 | _timers/batch_time=0.1747 | _timers/data_time=0.0652 | _timers/model_time=0.1095 | loss=0.3172 | mae_score=0.3453 | mse_score=0.3172 | qkappa_score=0.8743
1/5 * Epoch (train): 100% 202/202 [00:48<00:00,  4.94it/s, _timers/_fps=112.716, loss=0.025, mae_score=0.735, mse_score=0.878, qkappa_score=0.653]
1/5 * Epoch (valid): 100% 51/51 [00:13<00:00,  

5/15 * Epoch (train): 100% 202/202 [02:39<00:00,  1.35it/s, _timers/_fps=110.536, loss=0.125, mae_score=0.255, mse_score=0.163, qkappa_score=0.927]
5/15 * Epoch (valid): 100% 51/51 [00:13<00:00,  5.91it/s, _timers/_fps=249.393, loss=0.128, mae_score=0.228, mse_score=0.128, qkappa_score=0.800]
[2019-08-10 19:16:57,278] 
5/15 * Epoch 11 (train): _base/lr=0.0003 | _base/momentum=0.9000 | _timers/_fps=91.4320 | _timers/batch_time=0.1478 | _timers/data_time=0.0240 | _timers/model_time=0.1213 | loss=0.1338 | mae_score=0.3102 | mse_score=0.2725 | qkappa_score=0.8843
5/15 * Epoch 11 (valid): _base/lr=0.0003 | _base/momentum=0.9000 | _timers/_fps=128.8896 | _timers/batch_time=0.1578 | _timers/data_time=0.0708 | _timers/model_time=0.0869 | loss=0.4921 | mae_score=0.3108 | mse_score=0.4921 | qkappa_score=0.8996
6/15 * Epoch (train): 100% 202/202 [02:38<00:00,  1.35it/s, _timers/_fps=110.628, loss=0.056, mae_score=0.229, mse_score=0.077, qkappa_score=1.000]
6/15 * Epoch (valid): 100% 51/51 [00:13<

In [18]:
train_dataset = DiabeticDataset(dataset_path='/home/skolchen/kaggle/aptos/train/', 
                                labels = y_train, 
                                ids = X_train, 
                                albumentations_tr = aug_train(IMG_SIZE), 
                                extens='png') 
val_dataset = DiabeticDataset(dataset_path='/home/skolchen/kaggle/aptos/train/', 
                              labels = y_val, 
                              ids = X_val, 
                              albumentations_tr = aug_val(IMG_SIZE), 
                              extens='png') 
train_loader =  DataLoader(train_dataset,
                           num_workers=16,
                           pin_memory=False,
                           batch_size=12,
                           shuffle=True)
val_loader = DataLoader(val_dataset,
                        num_workers=16,
                        pin_memory=False,
                        batch_size=12)
loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = val_loader

In [21]:
for p in model.parameters():
    p.requires_grad = False
for p in model._fc.parameters():
    p.requires_grad = True
num_epochs = 3
optimizer = torch.optim.Adam(model.parameters(), lr=lr/10)
criterion = nn.MSELoss()
runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        logdir=logdir,
        callbacks=[
            QuadraticKappScoreMetricCallback(),
            MSECallback(),
            MAECallback(),               
                  ],
        num_epochs=num_epochs,
        verbose=True
        )     
for p in model.parameters():
    p.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=lr/10)
num_epochs = 7
criterion = nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=5)
runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        logdir=logdir,
        scheduler=scheduler,
        callbacks=[
            QuadraticKappScoreMetricCallback(),
            MSECallback(),
            MAECallback(),               
            EarlyStoppingCallback(patience=25, metric='loss')
                  ],
        num_epochs=num_epochs,
        verbose=True
        )        

0/3 * Epoch (train):  79% 159/202 [00:49<00:12,  3.53it/s, _timers/_fps=96.732, loss=0.006, mae_score=0.058, mse_score=0.006, qkappa_score=nan]  

/home/skolchen/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/sklearn/metrics/classification.py:373: RuntimeWarning:

invalid value encountered in true_divide



0/3 * Epoch (train): 100% 202/202 [01:01<00:00,  3.77it/s, _timers/_fps=114.039, loss=0.135, mae_score=0.214, mse_score=0.135, qkappa_score=0.957]
0/3 * Epoch (valid): 100% 51/51 [00:13<00:00,  5.93it/s, _timers/_fps=253.528, loss=0.105, mae_score=0.220, mse_score=0.105, qkappa_score=0.800]
[2019-08-10 19:48:26,409] 
0/3 * Epoch 22 (train): _base/lr=0.0000 | _base/momentum=0.9000 | _timers/_fps=93.2740 | _timers/batch_time=0.1453 | _timers/data_time=0.0246 | _timers/model_time=0.1206 | loss=0.1670 | mae_score=0.2309 | mse_score=0.1670 | qkappa_score=nan
0/3 * Epoch 22 (valid): _base/lr=0.0000 | _base/momentum=0.9000 | _timers/_fps=128.9171 | _timers/batch_time=0.1579 | _timers/data_time=0.0710 | _timers/model_time=0.0869 | loss=0.2177 | mae_score=0.2578 | mse_score=0.2177 | qkappa_score=0.9121
1/3 * Epoch (train): 100% 202/202 [01:01<00:00,  3.79it/s, _timers/_fps=114.278, loss=0.117, mae_score=0.170, mse_score=0.117, qkappa_score=0.921]
1/3 * Epoch (valid): 100% 51/51 [00:13<00:00,  5

/home/skolchen/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/sklearn/metrics/classification.py:373: RuntimeWarning:

invalid value encountered in true_divide



4/7 * Epoch (train): 100% 202/202 [02:38<00:00,  1.35it/s, _timers/_fps=111.823, loss=0.032, mae_score=0.100, mse_score=0.032, qkappa_score=1.000]
4/7 * Epoch (valid): 100% 51/51 [00:13<00:00,  5.94it/s, _timers/_fps=251.803, loss=0.083, mae_score=0.186, mse_score=0.083, qkappa_score=0.800]
[2019-08-10 20:06:24,743] 
4/7 * Epoch 30 (train): _base/lr=0.0000 | _base/momentum=0.9000 | _timers/_fps=92.5043 | _timers/batch_time=0.1435 | _timers/data_time=0.0217 | _timers/model_time=0.1217 | loss=0.1504 | mae_score=0.2136 | mse_score=0.1504 | qkappa_score=nan
4/7 * Epoch 30 (valid): _base/lr=0.0000 | _base/momentum=0.9000 | _timers/_fps=128.5990 | _timers/batch_time=0.1570 | _timers/data_time=0.0696 | _timers/model_time=0.0873 | loss=0.2393 | mae_score=0.2526 | mse_score=0.2393 | qkappa_score=0.9178
5/7 * Epoch (train): 100% 202/202 [02:38<00:00,  1.35it/s, _timers/_fps=113.263, loss=0.064, mae_score=0.187, mse_score=0.064, qkappa_score=1.000]
5/7 * Epoch (valid): 100% 51/51 [00:13<00:00,  5

/home/skolchen/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/sklearn/metrics/classification.py:373: RuntimeWarning:

invalid value encountered in true_divide



6/7 * Epoch (train): 100% 202/202 [02:38<00:00,  1.35it/s, _timers/_fps=111.663, loss=0.303, mae_score=0.274, mse_score=0.303, qkappa_score=0.952]
6/7 * Epoch (valid): 100% 51/51 [00:13<00:00,  5.92it/s, _timers/_fps=249.115, loss=0.089, mae_score=0.196, mse_score=0.089, qkappa_score=0.800]
[2019-08-10 20:12:20,666] 
6/7 * Epoch 32 (train): _base/lr=0.0000 | _base/momentum=0.9000 | _timers/_fps=92.4373 | _timers/batch_time=0.1451 | _timers/data_time=0.0234 | _timers/model_time=0.1216 | loss=0.1318 | mae_score=0.2004 | mse_score=0.1318 | qkappa_score=nan
6/7 * Epoch 32 (valid): _base/lr=0.0000 | _base/momentum=0.9000 | _timers/_fps=127.9897 | _timers/batch_time=0.1583 | _timers/data_time=0.0712 | _timers/model_time=0.0870 | loss=0.2343 | mae_score=0.2501 | mse_score=0.2343 | qkappa_score=0.9188
Top best models:
logs/efficient_net_b7_trainregression_smart_v1/checkpoints//train.26.pth	0.2210
logs/efficient_net_b7_trainregression_smart_v1/checkpoints//train.28.pth	0.2241
logs/efficient_net

In [19]:
from catalyst.dl import utils

In [22]:
logdir = './logs/efficient_net_b7_trainregression_smart_v1/'
utils.plot_metrics(logdir=logdir, metrics=['loss','qkappa_score','lr','mae_score','mse_score'])